In [128]:
import math

from solid import *
from solid.utils import *

from subprocess import run

import viewscad
r = viewscad.Renderer()

In [129]:
import os, solid; print(os.path.dirname(solid.__file__) + '/examples')

/home/flo/.local/lib/python3.9/site-packages/solid/examples


In [130]:
d = cube([7,4,4]) + sphere(5) - cylinder(r=2, h=6)
r.render(d)

In [131]:
# CONFIGURATION
GENERATE_WITH_KEYCAPS = False
HOLE_HEIGHT_TOLERANCE = 0.2 # 0 gives weird viewing glitches

In [132]:
KEYCAP_SIZE = 18
KEYCAP_HEIGHT = 9
KEYCAP_Z_OFFSET = 6.6

PLATE_GRID = 19.05
PLATE_HOLE = 14
PLATE_THICKNESS = 4
# PLATE_SPACER = SWITCH_GRID - PLATE_HOLE  # = 5.05mm

def plate_cutout(units_x=1, units_y=1, withCap=False, center=True):
    offset = units_x - 1;
    plate = cube([PLATE_GRID*units_x,PLATE_GRID*units_y,PLATE_THICKNESS], center=True)
    hole = cube([PLATE_HOLE,PLATE_HOLE,PLATE_THICKNESS+HOLE_HEIGHT_TOLERANCE], center=True)
    cutout = plate - hole
    
    # TODO: if units_x or units_y >= 2: add stab holes

    if(withCap):
        cap = color([0.0,1.0,0.5])(cube([KEYCAP_SIZE*units_x,KEYCAP_SIZE*units_y,KEYCAP_HEIGHT], center=True))
        cap = up(PLATE_THICKNESS/2 + KEYCAP_HEIGHT/2 + KEYCAP_Z_OFFSET)(cap)
        cutout += cap
    
    if(center == False):
        cutout = left(units_x*PLATE_GRID/2)(back(units_y*PLATE_GRID/2)(cutout))
    
    return cutout

r.render(plate_cutout(units_x=1, units_y=2, withCap=True, center=False))

In [133]:
# keys[units_y], units_x
def plate_column(keys=[1,1,1,1], units_x=1, center=True):
    i,offset,column = 0,0,cube(0)
    while(i<len(keys)):
        units_y = keys[i]
        cutout = plate_cutout(units_x=units_x, units_y=units_y, withCap=GENERATE_WITH_KEYCAPS)
        cur_offset = units_y/2
        column += forward(PLATE_GRID*(offset + cur_offset))(cutout)
        offset += (units_y/2) + cur_offset
        i += 1
    if(center == False):
        column = right(units_x*PLATE_GRID/2)(column)
    return column
        
r.render(plate_column(keys=[2,1,2,1], units_x=1.5, center=False))

In [134]:
# [column[keys[],units,y_offset_units]]
# keep the y_offset_units in multiple or halfs of 1u -> for easier alignment in KiCAD later

def plate(layout):
    i,cur_offset,offset,plate = 0,0,0,cube(0)
    while(i<len(layout)):
        column_specs = layout[i]
        column = plate_column(keys=column_specs[0],units_x=column_specs[1], center=True)
        cur_offset = column_specs[1]/2
        plate += right(PLATE_GRID*(offset+cur_offset))(forward(PLATE_GRID*column_specs[2])(column))
        offset += column_specs[1]/2 + cur_offset
        i += 1
    return plate
    

r.render(plate([[[2,1,1.5,1],1.5,0],[[1,1,1,1.5],1,0.5],[[1,1,1,1,2],2,-0.5],[[1,2,1],1,0]]))

In [135]:
C4 = [1,1,1,1]
C5 = [1,1,1,1,1]
C3_vert = [1.5,1.5,1]
layout = [[C4,1.5,0],[C4,1,0],[C5,1,-0.5],[C5,1,-0.375],[C5,1,-0.5],[C4,1,0.25],[C3_vert,1,0.25]]

myPlate = plate(layout)

space_angle = math.radians(12) # TODO den nicesten Winkel finden ( ͡° ͜ʖ ͡°)
space_units = 1.5

thumbcluster_keys = [1,1]

# predefined by layout:
space_offset_columns = 5
space_offset_y = 0.75

# get amount of units left after the offset
i = space_offset_columns
space_offset_available = 0
while(i < len(layout)):
    space_offset_available += layout[i][1]
    i += 1

space = plate_cutout(units_x=space_units, center=False)
space = right(PLATE_GRID*space_units)(space) # set anchor on top right edge
thumbcluster = plate_column(keys=thumbcluster_keys, units_x=1, center=False)

if(space_angle >= math.pi/4):
    # idk why I calculated that one as well
    adjacent = math.cos(math.pi/2-space_angle)*PLATE_GRID
    opposite = math.sin(math.pi/2-space_angle)*PLATE_GRID
    # this probably doesn't work as expected. I realised too late that this isn't even the angle I wanted
    space_edge_y = opposite + space_units*PLATE_GRID/math.sin(space_angle)
    space_edge_x = (space_edge_y-opposite)/math.tan(space_angle)
else:
    # this is the working and tested version of this code
    adjacent = space_offset_y*PLATE_GRID/math.tan(math.pi/2-space_angle)
    opposite = space_offset_y*PLATE_GRID
    hypotenuse = math.sqrt(pow(adjacent,2) + pow(opposite,2))
    
    opposite_offset = (PLATE_GRID-hypotenuse)/math.cos(space_angle)
    adjacent_offset = (PLATE_GRID-hypotenuse)*math.sin(space_angle)
    
    space_edge_y = opposite + opposite_offset + space_units*PLATE_GRID*math.sin(space_angle)
    space_edge_x = space_units*PLATE_GRID*math.cos(space_angle) - adjacent_offset


# position the thumbcluster
angle_space_thumbcluster = math.atan((space_offset_available*PLATE_GRID - space_edge_x)/space_edge_y) - space_angle
if angle_space_thumbcluster < 0:
    angle_space_thumbcluster = 0

filler_y = PLATE_GRID*math.tan(angle_space_thumbcluster)
filler_points = [
    [0,0,-PLATE_THICKNESS/2],
    [filler_y,PLATE_GRID,-PLATE_THICKNESS/2],
    [0,PLATE_GRID,-PLATE_THICKNESS/2],
    [0,0,PLATE_THICKNESS/2],
    [filler_y,PLATE_GRID,PLATE_THICKNESS/2],
    [0,PLATE_GRID,PLATE_THICKNESS/2]
]
filler_faces = [[0,1,2],[0,1,3,4],[1,2,4,5],[2,0,5,3],[5,4,3]] # do I need hidden faces for the slicer?
filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5)
space += back(PLATE_GRID)(right(space_units*PLATE_GRID)(filler))

thumbcluster = rotate(a=math.degrees(angle_space_thumbcluster), v=DOWN_VEC)(thumbcluster) # rotate
thumbcluster = back(PLATE_GRID)(right(space_units*PLATE_GRID)(thumbcluster)) # move next to space
space += thumbcluster

filler_points = [
    [0,0,-PLATE_THICKNESS/2],
    [adjacent,0,-PLATE_THICKNESS/2],
    [0,-opposite,-PLATE_THICKNESS/2],
    [0,0,PLATE_THICKNESS/2],
    [adjacent,0,PLATE_THICKNESS/2],
    [0,-opposite,PLATE_THICKNESS/2]
]
filler_faces = [[0,1,2],[0,1,3,4],[1,2,4,5],[2,0,5,3],[5,4,3]] # do I need hidden faces for the slicer?
filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5)

space = rotate(a=math.degrees(space_angle), v=DOWN_VEC)(space) # rotate

i,space_offset_x,space_offset_y = 0,0,layout[space_offset_columns][2]
while i<space_offset_columns:
    space_offset_x += layout[i][1]
    i += 1

myPlate += right(space_offset_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))
myPlate += right(space_offset_x*PLATE_GRID + adjacent)(forward(space_offset_y*PLATE_GRID)(space))

filler_x = space_units*PLATE_GRID*math.cos(space_angle) + adjacent
filler_y = -space_units*PLATE_GRID*math.sin(space_angle)
filler_points = [
    [adjacent,0,-PLATE_THICKNESS/2],
    [filler_x,filler_y,-PLATE_THICKNESS/2],
    [space_offset_available*PLATE_GRID,0,-PLATE_THICKNESS/2],
    [adjacent,0,PLATE_THICKNESS/2],
    [filler_x,filler_y,PLATE_THICKNESS/2],
    [space_offset_available*PLATE_GRID,0,PLATE_THICKNESS/2]
]
filler_faces = [[0,1,2],[0,1,3,4],[1,2,4,5],[2,0,5,3],[5,4,3]] # do I need hidden faces for the slicer?
filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5)
myPlate += right(space_offset_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))

r.render(myPlate)

In [136]:
 projection(d) # muss ich vlt doch in openscad machen

In [137]:
# generate valid openscad code and store it in file
scad_render_to_file(myPlate, 'plate.scad')

# run openscad and export to stl
run(["openscad", "-o",  "plate.stl", "plate.scad"])

CompletedProcess(args=['openscad', '-o', 'plate.stl', 'plate.scad'], returncode=0)

I should have probably gone with a vectore based approach…